<a href="https://colab.research.google.com/github/vadim-privalov/Neiroset_Novosibirsk/blob/main/%D0%9F%D0%BE%D0%B4%D1%81%D1%87%D1%91%D1%82_%D0%BB%D1%8E%D0%B4%D0%B5%D0%B9_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_opencv_%D0%B8_dlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="text-align: center;"><b>Подсчёт людей с помощью opencv и dlib</b></h1>


<h1 style="text-align: center;"><b>Counting people with opencv and dlib</b></h1>

В этом уроке мы будем считать сколько людей пересекли линию, сколько прошли вверх и вниз. Находить и выделять людей мы будем с помощью opencv используя сеть MobileNet SSD, а отслеживать их перемещение будем с помощью корреляционного трекера с библиотеки dlib. Не ждите очень хорошего качества выделения и отслеживания объектов. Сеть MobileNet SSD относительно простая и корреляционный трекер не всегда хорошо обрабатывает окклюзии.

In this tutorial we will count how many people crossed the line, how many went up and down. We will find and select people with opencv using the MobileNet SSD network, and track their movement with the correlation tracker from the dlib library. Don't expect very good extraction and tracking quality. The MobileNet SSD network is relatively simple and the correlation tracker doesn't always handle occlusions well.


Это более простой урок для демонстрации работы принципа, по которому происходит посчёт людей. В будущих уроках мы усовершенствуем эту схему и будем использовать более мощные и точные иструменты.




This is a simpler lesson to demonstrate how the principle behind how people are counted works. In future lessons, we will improve this scheme and use more powerful and accurate tools.


Скачиваем файлы, необходимые для урока, и устанавливаем корневую папку проекта.

Download the files needed for the lesson, and install the root folder of the project.


In [1]:
!wget http://dataudt.ru/datasets/cv/Lesson_50.Counting_people_opencv_dlib.zip
!unzip -qq Lesson_50.Counting_people_opencv_dlib.zip
%cd /content/Lesson_50.Counting_people_opencv_dlib/

--2022-02-20 09:53:00--  http://dataudt.ru/datasets/cv/Lesson_50.Counting_people_opencv_dlib.zip
Resolving dataudt.ru (dataudt.ru)... 37.228.117.130
Connecting to dataudt.ru (dataudt.ru)|37.228.117.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26920482 (26M) [application/zip]
Saving to: ‘Lesson_50.Counting_people_opencv_dlib.zip’

Lesson_50.Counting_ 100%[===================>]  25.67M  12.8MB/s    in 2.0s    

2022-02-20 09:53:03 (12.8 MB/s) - ‘Lesson_50.Counting_people_opencv_dlib.zip’ saved [26920482/26920482]

/content/Lesson_50.Counting_people_opencv_dlib


Импортируем необходимые модули.

Import the necessary modules.


In [2]:
# дополнительные модули для учёта и отслеживания объектов
# additional modules for object accounting and tracking
from Objects.centroidtracker import CentroidTracker
from Objects.trackableobject import TrackableObject
# библиотека для корреляционного трекера
# library for the correlation tracker
import dlib
# модуль для отображения картинок в colab
# module for displaying pictures in colab
from google.colab.patches import cv2_imshow

import numpy as np
import cv2
import time

Укажем путь к модели, входным видео, и зададим параметры.

Let's specify the path to the model, the input video, and set the parameters.


In [3]:
args = {
    "prototxt": "Model/MobileNetSSD_deploy.prototxt", # параметры модели
                                                        # model parameters
    "model": "Model/MobileNetSSD_deploy.caffemodel", # модель нейронной сети
                                                    # neural network model
    "input": "Videos_in/example_02.mp4", # путь входного видео
                                        # input video path
    "output": "Videos_out/output_02.avi", # путь для выходного видео
                                        # way for the output video
    "confidence": 0.4, # уровень достоверности обнаружения объектов
                        # level of object detection reliability
    "skip_frames": 20  # количество кадров, после которого происходит обнаружение
                    # number of frames after which detection occurs
}

Укажем координаты точек линии подсчёта. И зададим функцию для расчёта точек пересечения линии.

Specify the coordinates of the counting line points. And set a function to calculate the intersection points of the line.

In [4]:
line_x1, line_y1 = 45, 158
line_x2, line_y2 = 550, 335

def border_line(x):
    return int((x-line_x1)/(line_x2-line_x1)*(line_y2-line_y1)+line_y1)

In [5]:
# инициализируем список меток классов, которые
# MobileNet SSD была обучена обнаруживать
# initialize the list of class labels MobileNet SSD was trained to
# detect
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]

# загружаем нашу сохранённую модель с диска
# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

[INFO] loading model...


In [6]:
# используем ссылку на видео файл
# и захватываем видео
# grab a reference to the video file
print("[INFO] opening video file...")
vs = cv2.VideoCapture(args["input"])

# инициализируем средство записи видео 
# (при необходимости мы создадим его экземпляр позже)
# initialize the video writer (we'll instantiate later if need be)
writer = None

# инициализируем размеры кадра 
# (мы установим их, как только прочитаем первый кадр из видео)
# initialize the frame dimensions (we'll set them as soon as we read
# the first frame from the video)
W = None
H = None

[INFO] opening video file...


In [7]:
# создаем экземпляр нашего трекера центроидов, 
# затем инициализируем список для хранения каждого из наших 
# трекеров корреляции dlib, за которым следует словарь для 
# сопоставления каждого уникального идентификатора объекта
# instantiate our centroid tracker, then initialize a list to store
# each of our dlib correlation trackers, followed by a dictionary to
# map each unique object ID to a TrackableObject
ct = CentroidTracker(maxDisappeared=40, maxDistance=50)
trackers = []
trackableObjects = {}

# инициализируем общее количество кадров, обработанных 
# на данный момент, вместе с общим количеством объектов,
# которые переместились вверх или вниз
# initialize the total number of frames processed thus far, along
# with the total number of objects that have moved either up or down
totalFrames = 0
totalDown = 0
totalUp = 0

In [8]:
%%time
# засечём время для оценки fps 
# time to estimate fps 
start_time = time.time()

# перебираем кадры из видеопотока
# loop over frames from the video stream
while True:
    # считываем кадр с видео
    # grab the next frame 
    success, frame = vs.read()
    
    if success:
        # конвертируем кадр из BGR в RGB для dlib
        # convert the frame from BGR to RGB for dlib
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # если размеры фрейма пустые, установим их
        # if the frame dimensions are empty, set them
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        # инициализируем запись
        # initialize the writer
        if args["output"] is not None and writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(args["output"], fourcc, 30,
                (W, H), True)

        # инициализируем текущий статус вместе с нашим списком 
        # ограничивающих рамок, возвращаемыми либо (1) детектором
        # объектов, либо (2) трекерами корреляции
        # initialize the current status along with our list of bounding
        # box rectangles returned by either (1) our object detector or
        # (2) the correlation trackers
        status = "Waiting"
        rects = []

        # проверяем, следует ли запускать более дорогостоящий в 
        # вычислительном отношении метод обнаружения объектов,
        # чтобы помочь нашему трекеру
        # check to see if we should run a more computationally expensive
        # object detection method to aid our tracker
        if totalFrames % args["skip_frames"] == 0:
            # устанавливаем статус и инициализируем 
            # наш новый набор трекеров объектов
            # set the status and initialize our new set of object trackers
            status = "Detecting"
            trackers = []

            # преобразовываем кадр, пропускаем его через сеть и получаем обнаружение
            # convert the frame to a blob and pass the blob through the
            # network and obtain the detections
            blob = cv2.dnn.blobFromImage(frame, 0.007843, (W, H), 127.5)
            net.setInput(blob)
            detections = net.forward()

            # перебираем что обнаружили
            # loop over the detections
            for i in np.arange(0, detections.shape[2]):
                # извлекаем достоверность (т.е. вероятность), связанную с прогнозом
                # extract the confidence (i.e., probability) associated
                # with the prediction
                confidence = detections[0, 0, i, 2]

                # отфильтровываем обнаружения меньше
                # минимальной достоверности
                # filter out weak detections by requiring a minimum
                # confidence
                if confidence > args["confidence"]:
                    # извлекаем индекс метки класса из списка обнаружений
                    # extract the index of the class label from the
                    # detections list
                    idx = int(detections[0, 0, i, 1])

                    # если метка класса не человек, игнорируем её
                    # if the class label is not a person, ignore it
                    if CLASSES[idx] != "person":
                        continue

                    # вычисляем x, y координаты ограничивающего 
                    # прямоугольника объекта
                    # compute the (x, y)-coordinates of the bounding box
                    # for the object
                    box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
                    (startX, startY, endX, endY) = box.astype("int")

                    # построим dlib прямоугольный объект из координат 
                    # ограничивающего прямоугольника, а затем 
                    # запустим трекер корреляции dlib
                    # construct a dlib rectangle object from the bounding
                    # box coordinates and then start the dlib correlation
                    # tracker
                    tracker = dlib.correlation_tracker()
                    rect = dlib.rectangle(startX, startY, endX, endY)
                    tracker.start_track(rgb, rect)

                    # добавляем трекер в наш список трекеров, чтобы мы могли 
                    # использовать его во время пропуска кадров
                    # add the tracker to our list of trackers so we can
                    # utilize it during skip frames
                    trackers.append(tracker)

        # в противном случае мы должны использовать наши объектные трекеры, 
        # а не объектные детекторы, чтобы получить более высокую 
        # пропускную способность обработки кадров
        # otherwise, we should utilize our object *trackers* rather than
        # object *detectors* to obtain a higher frame processing throughput
        else:
            # перебираем трекеры
            # loop over the trackers
            for tracker in trackers:
                # установим статус нашей системы как «отслеживание», 
                # а не «ожидание» или «обнаружение»
                # set the status of our system to be 'tracking' rather
                # than 'waiting' or 'detecting'
                status = "Tracking"

                # обновим трекер и получим обновленную позицию
                # update the tracker and grab the updated position
                tracker.update(rgb)
                pos = tracker.get_position()

                # распаковываем координаты позиции объекта 
                # unpack the position object
                startX = int(pos.left())
                startY = int(pos.top())
                endX = int(pos.right())
                endY = int(pos.bottom())

                # добавляем координаты ограничивающего прямоугольника 
                # в список прямоугольников
                # add the bounding box coordinates to the rectangles list
                rects.append((startX, startY, endX, endY))

        # рисуем горизонтальную линию в центре кадра 
        # как только объект пересекает эту линию, мы определяем, 
        # двигался ли он вверх или вниз
        # draw a horizontal line in the center of the frame -- once an
        # object crosses this line we will determine whether they were
        # moving 'up' or 'down'
        cv2.line(frame, (line_x1, line_y1), (line_x2, line_y2), (0, 255, 255), 2)

        # используем трекер центроидов, чтобы связать (1) старые центроиды
        # объектов с (2) вновь вычисленными центроидами объектов
        # use the centroid tracker to associate the (1) old object
        # centroids with (2) the newly computed object centroids
        objects = ct.update(rects)

        # перебираем отслеживаемые объекты
        # loop over the tracked objects
        for (objectID, centroid) in objects.items():
            # проверим, существует ли отслеживаемый объект 
            # для текущего идентификатора объекта
            # check to see if a trackable object exists for the current
            # object ID
            to = trackableObjects.get(objectID, None)

            # если нет существующего отслеживаемого объекта, создим его
            # if there is no existing trackable object, create one
            if to is None:
                to = TrackableObject(objectID, centroid)

            # в противном случае существует отслеживаемый объект, поэтому 
            # мы можем использовать его для определения направления
            # otherwise, there is a trackable object so we can utilize it
            # to determine direction
            else:
                # разница между координатой y текущего центроида и средним 
                # значением предыдущих центроидов укажет нам, в каком 
                # направлении движется объект 
                # (отрицательная для 'вверх' и положительная для 'вниз')
                # the difference between the y-coordinate of the *current*
                # centroid and the mean of *previous* centroids will tell
                # us in which direction the object is moving (negative for
                # 'up' and positive for 'down')
                y = [c[1] for c in to.centroids]
                direction = centroid[1] - np.mean(y)
                to.centroids.append(centroid)

                # проверяем, посчитан ли объект
                # check to see if the object has been counted or not
                if not to.counted:
                    # если направление отрицательное (указывает на то, 
                    # что объект движется вверх) и центроид
                    #  выше центральной линии, то считаем объект
                    # if the direction is negative (indicating the object
                    # is moving up) AND the centroid is above the center
                    # line, count the object
                    border_y = border_line(centroid[0])
                    if direction < 0 and centroid[1] < border_y:
                        totalUp += 1
                        to.counted = True

                    # если направление положительное (указывает на то, 
                    # что объект движется вниз) и центроид 
                    # ниже центральной линии, то считаем объект
                    # if the direction is positive (indicating the object
                    # is moving down) AND the centroid is below the
                    # center line, count the object
                    elif direction > 0 and centroid[1] > border_y:
                        totalDown += 1
                        to.counted = True

            # сохраняем отслеживаемый объект в нашем словаре
            # store the trackable object in our dictionary
            trackableObjects[objectID] = to

            # рисуем как ID объекта, так и 
            # центроид объекта на выходном кадре
            # draw both the ID of the object and the centroid of the
            # object on the output frame
            text = "ID {}".format(objectID)
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

        # создаем кортеж информации, который мы будем отображать в кадре
        # construct a tuple of information we will be displaying on the
        # frame
        info = [
            ("Up", totalUp),
            ("Down", totalDown),
            ("Status", status),
        ]

        # перебираем информационные кортежи и рисуем их на нашем фрейме
        # loop over the info tuples and draw them on our frame
        for (i, (k, v)) in enumerate(info):
            text = "{}: {}".format(k, v)
            cv2.putText(frame, text, (10, H - ((i * 20) + 20)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

        # записываем выходной кадр в видео
        # record the output frame in the video
        writer.write(frame)

        # Не будем отображать выходное изображение
        # так как colab не выводит поток как видео,
        # а отображает ленту картинок, что очень
        # снижает производительность.
        # Чтобы посмотреть как отработала наша 
        # программа воспользуемся кодом ниже
        # cv2_imshow(frame)
        # Will not display the output image
        # since colab does not output the stream as video,
        # but displays a feed of images, which is very
        # decreases performance.
        # To see how our
        # program we will use the code below
        # cv2_imshow (frame)

        # increment the total number of frames processed thus far 
        # увеличиваем общее количество кадров, 
        # обработанных на данный момент
        # increment the total number of frames processed thus far
        # increase the total number of frames,
        # currently processed
        totalFrames += 1
    else:
        break

# оценим производительность обработки в кадрах в секунду
# let's estimate the processing performance in frames per second
print('FPS: {:.2f}'.format(totalFrames/(time.time()-start_time)))

FPS: 21.00
CPU times: user 41 s, sys: 270 ms, total: 41.2 s
Wall time: 29.3 s


Чтобы посмотреть выходное видео непосредственно в Colab, установим модуль kora.

To view the output video directly in Colab, install the kora module.


In [9]:
!pip install -U kora

     |████████████████████████████████| 57 kB 2.9 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 


Kora может восипроизводить видео только в формате .mp4, а выходные видео нашей программы в формате .avi. Конвертируем формат видео. Конвертация может занять какое-то время.


Kora can only produce videos in .mp4 format, and our program's output videos are in .avi format. Convert the video format. Conversion may take some time.

Для этого воспользуемся командой ffmpeg, укажем путь к выходному видео в .avi формате, и укажем имя, и путь для сконвертированного видео. (Если сконвертированное видео уже существует с таким именем, то нужно будет подтвердить замену указав "y" yes в поле.)


To do this, use the command ffmpeg, specify the path to the output video in .avi format, and specify the name and path for the converted video. (If the converted video already exists with that name, you will need to confirm the replacement with "y" yes in the field.)

In [10]:
!ffmpeg -i Videos_out/output_02.avi Videos_out/output_02.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

Воспроизведём сконвертированное видео в Colab. Для этого дадим разрешение Google Cloud SDK на доступ к файлам.

Play the converted video in Colab. To do this, give Google Cloud SDK permission to access the files.

In [11]:
from kora.drive import upload_public
url = upload_public('Videos_out/output_02.mp4')

from IPython.display import HTML
HTML(f"""<video src={url} width=640 controls/>""")